In [ ]:
import zmq
import numpy as np
import cv2
import io
import timeit

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.python.ops.numpy_ops import np_config
from PIL import Image

np_config.enable_numpy_behavior()

In [ ]:
# Parameters to change !!!

generatorPath = 'generator_heightmap.h5'
socketBind = 'tcp://*:5555'

In [ ]:
print("Server starting...");

In [ ]:
print("Loading model...")
model = load_model(generatorPath)

In [ ]:
print("Model loaded.")
print("Performing a preliminary prediction...")
tmpArray = np.zeros(shape=(1, 512, 512, 3))
tmpData = model(tmpArray)

In [ ]:
print("Establishing a connection...")
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind(socketBind)

In [ ]:
def load_image(pixels):
    pixels = img_to_array(pixels)
    pixels = (pixels - 127.5) / 127.5
    pixels = np.expand_dims(pixels, 0)
    
    return pixels

In [ ]:
print("Ready. Waiting for data.")

while True:
    bytes_received = socket.recv(zmq.CONFLATE)
    print('Received sketch. Processing...')
    start = timeit.default_timer()
    image = Image.open(io.BytesIO(bytes_received))
    image = load_image(image)
    
    gen_image = model(image)
    gen_image = (gen_image + 1) / 2.0    
    
    predicted_map = (gen_image[0] * 255.0).astype(np.uint8)   
    
    predicted_map = cv2.medianBlur(np.array(predicted_map),5)
    bytes_to_send = predicted_map.tobytes()
       
    socket.send(bytes_to_send)
    stop = timeit.default_timer()
    print('Created heightmap. Time [s]: ', stop - start) 